문장에서 다음 글자를 예측하는 문자 단위 RNN
상태가 없는 RNN (stateless RNN) 무작위로 선택한 텍스트이 일부분만 학습
상태가 있는 RNN (stateful RNN) 

디코더시 확률 출력
확률적으로 더 정확한 추론을 할 수 있다



## 1.1 훈련 데이터셋

1. Dataset download

In [3]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if IS_COLAB:
    %pip install -q -U tensorflow-addons
    %pip install -q -U transformers

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

2023-04-09 18:13:31.488610: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


In [4]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


2. 모든 글자 -인코딩-> 정수로

방법1. 사용자 정의 전처리 층 만들기
방법2. keras - Tokenizer class 사용

**<keras - Tokenizer 사용>**    
Tokenizer 클래스에 텍스트 training
텍스트 소문자로 다 바꿔줌 - lower=False로 막을 수 있음     
텍스트 안의 모든 character 글자에 각 ID를 매핑.    
ID는 1 ~ . (마스킹 사용하려면 0 사용안해야해서) 

char_level=True : 글자 수준 인코딩 진행    
char_level=False : 단어 수준 인코딩 진행

In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [6]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [8]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count
print("고유 글자 개수 : ",max_id, ", 텍스트의 글자 개수 : ",dataset_size)

고유 글자 개수 :  39 , 텍스트의 글자 개수 :  1115394


[encoded]에 훈련된 tokenizer를 이용해 텍스트를 정수로 변환해주고,    
numpy array로 데이터 타입을 변환한 것을 선언해준다.

## 1.2 순차 데이터셋을 나눈다

train_size에는 텍스트 데이터를 train, val, test data로 나누기 위해 전체 중 90%의 숫자를 구한 것을 볼 수 있다.

dataset 에는 구한 train_size를 이용하여 데이터셋 중 train data를 구해준다.

In [13]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
print("[encoded] ",[encoded], "\n train_size", train_size, "\n dataset",dataset)

[encoded]  [array([19,  5,  8, ..., 20, 26, 10])] 
 train_size 1003854 
 dataset <TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>


여기서는 전체 중 90%를 train data로 사용했지만, 시계열의 경우 보통 시간에 따라 나눈다. 
ex) 2000y ~ 2012y (train) 2013y ~ 2015y (val)    
2016y ~ 2018y (test)

그 이유는 데이터셋들은 서로 연관성이 없는 것이 좋은데, 만약 train 데이터와 test 데이터가 상호 연관이 있다면 test predict를 할 때 일반 오차가 낙관적으로 -> 편향될 수 있다.

문제에 따라서 그 기준을 바꿀수가 있다.

## 1.3 순차 데이터를 윈도 여러 개로 자르기
- 훈련 세트 = 백만 개 글자의 시퀀스 1개 -> 신경망을 직접 훈련시킬 수 없음    
- window() method로 긴 시퀀스를 작은 많은 텍스트 윈도로 변환하여 훈련 가능.

- 0에서 window_length번째 글자까지의 데이터셋 윈도우 1개를 만든다.    
- 그리고 shift = 1 , 1칸씩 뒤로 이동하면서 (window_length + 1)개의 윈도우 1개를 또 만든다.
- drop_remainder=True로 지정하면 모든 윈도우의 글자 개수가 (window_length + 1) 개로 동일하다. 마지막 데이터셋이 포함될 때 까지 윈도우를 만든다.    
- 만약, drop_remainder=False 이면 100개, 99개, ... 1개로 데이터셋의 개수가 줄어들며 윈도우가 만들어질 것이다.    

In [14]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [15]:
dataset

<WindowDataset element_spec=DatasetSpec(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorShape([]))>

dataset type = WindowDataset

model training하려면 텐서 타입이어야 한다. - flat_map()을 사용하여 
{{1,2},{3,4,5}} 의 형태를 -> {1,2,3,4,5} 로 변경해준다.

그리고 flat_map()은 함수도 받아서 처리할 수 있다.

In [16]:
# 101개의 글자를 담은 윈도
dataset = dataset.flat_map(lambda window: window.batch(window_length))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


경사 하강법이 동일 독립 분포일 때 잘 작동하므로 윈도를 섞어서 데이터가 다양하도록 해줘야 한다.

In [18]:
np.random.seed(42)
tf.random.set_seed(42)

# window 크기 101, batch size 32
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
#                                            train_x        train_y

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


범주형 -> onehot encode / embedding encode    

고유한 글자 수가 적은 경우 -> onehot encode

In [19]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [20]:
dataset = dataset.prefetch(1) 
# cpu 가까운 곳에 데이터를 미리 가져다 놓아서 데이터를 가져오는 동안 cpu가 노는 것을 방지하여
# 계산이 빠르게 진행되게 함

In [21]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


## 1.4 Char-RNN Model 만들고 Training

#### 모델 설계하기
글자 100개로 다음 글자 예측     
유닛 128개    
GRU 2개 layer    
dropout, recurrent_dropout 20% 적용    
고유한 글자 수 39개 -> max_id 39개    
타임 스텝마다 각 글자에 대한 확률 출력 -> 출력 합 1.0 이어야함 -> softmax 적용    
손실 sparse_categorical_crossentropy, optimizer Adam 사용    
compile()    

n번의 epoch 동안 모델 훈련

In [23]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
                    #  dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
                    #  dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=1)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
   3859/Unknown - 930s 239ms/step - loss: 1.7220

KeyboardInterrupt: 

## 1.5 Char-RNN Model 사용하기
앞에서 만든 모델을 사용해보자.

#### 앞에서 "셰익스피어가 쓴 텍스트에서 다음 글자를 예측하는 모델"을 만들었다.
이 모델에 새로운 텍스트 데이터를 주입하는 방법 (예측하는 방법인가?)

텍스트 데이터를 앞과 같이 전처리하고 모델에 적용시킨다.

In [24]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [29]:
X_new = preprocess(["How are yo"])
# Y_pred = model.predict_classes(X_new)
# AttributeError: 'Sequential' object has no attribute 'predict_classes'
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'u'

In [30]:
# tf.random.set_seed(42)
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[1, 0, 1, 1, 1, 0, 2, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 1, 1, 1, 0, 0, 2, 0, 1, 1, 0, 0, 0, 2, 0, 1, 1]])

categorical() 함수는 각 클래스별 로그 확률값을 이용하여 랜덤하게 클래스 인덱스를 샘플링한다.

temperature 값으로 로그 확률값을 나누게 되는데, 이 값이 크면 클수록 제약이 커지게 되어 모든 글자가 동일하게 나타날 수 있다. 0값에 가까울수록 높은 확률을 가진 글자를 선택한다.

아래의 next_char() 함수는 다음 글자를 선택하여 입력 텍스트에 추가하는 것이다.

In [33]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature # log 적용
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1 # 각 카테고리별 확률
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [34]:
next_char("How are yo", temperature=1) # 제작한 함수로 나의 텍스트 다음에 나올 것 예측

'u'

모델을 이용하여 텍스트를 생성하기 위해서 텍스트가 추가될 수 있는 함수를 만들어준다.

In [35]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [39]:
print("0.2:",complete_text("t", temperature=0.2))
print("1:",complete_text("t", temperature=1))
print("2:",complete_text("t", temperature=2))

0.2: the country him.

sicinius:
what he was and the wor
1: t him, let's are rome in thee hast have
me, and bot
2: t-vaip.
'reivaty! co tazedcason:
he womd, to con of


위 결과를 보면, temperature의 값이 1일때 가장 잘 작동하는 것이라고 한다.

더 좋은 성능을 내기 위해서는, GRU layer, neuron layer 를 추가 + epoch을 추가 + 규제 추가(recurrent_dropout=0.3 설정 같은) 해볼 수 있다.

매우 긴 시퀀스 : LSTM, GRU 는 힘들다. stateful RNN 시도 해볼 것

## 1.7 Stateful RNN
상태가 있는 RNN

stateless RNN -> 훈련 반복마다 모델의 은닉 상태를 0으로 초기화. 한 타임 스텝 후 초기화.    
훈련 배치 후 마지막 상태를 다음 배치의 초기값으로 사용하면? 장기간 패턴 학습 가능 -> stateful RNN

각 input sequence가 이전 batch의 sequence 끝난 지점에서 시작해야 함.    
그러기 위해서 1. 순차적, 중복되지 않는 input sequence 생성     
stateful에서는 shuffle() 안씀, shift=1 대신 shift=n_steps를 씀.    
batch 하기 힘듬(왜냐면 shuffle 안해서 32개의 연속적인 윈도가 같은 배치에 들어가기 때문)    
1~32, 33~63 배치가 진행된다 ... 이러한 문제는 ... batch(1)로 만든다...    

In [40]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the ver

텍스트 데이터를 모두 32개씩 나누고, 각 텍스트에 대해 연속적인 입력 시퀀스를 가진 데이터셋 하나를 만든다.    
그리고 연속적인 배치를 두어 데이터셋 하나를 만들 수 있다.

In [41]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
# 연속적인 배치 생성
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

stateful=True 로 설정할 것.    
batch_input_shape 알아야함. 입력의 길이는 제한이 없으므로 지정하지 None 않아도 됨.


In [42]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2, # GPU 사용 가능
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

epoch 마다 상태 재설정해줘야함 -> callback 함수 사용

In [43]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

Model compile & training Step

In [44]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=1,
                    callbacks=[ResetStatesCallback()])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
313/313 [==============================] - 50s 149ms/step - loss: 2.6231


상단은 **글자 수준 모델** 생성 단계였다.    

이제 __*단어 수준 모델*__을 살펴보고 감성 분석을 다룬다. 마스킹을 사용해 길이가 다른 시퀀스도 다룰 수 있다.

# 2. 감성 분석

IMDb 리뷰 데이터셋 이용한 실습    
리뷰와 부정 0 긍정 1 이진 타깃인 데이터

In [46]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

17464789/17464789 [==============================] - 1s 0us/step
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Users/gahyunson/opt/anaconda3/envs/mywork/lib/python3.7/site-packages/keras/backend.py", line 5134, in <genexpr>
    for ta, out in zip(output_ta_t, flat_output)  File "/Users/gahyunson/opt/anaconda3/envs/mywork/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py", line 245, in wrapped
    error_in_function=error_in_function)


In [48]:
""" 
낮은 정수가 자주 등장하는 단어임 
텍스트 구두점 제거, 소문자로 모두 변환, 공백 기준으로 나누고, 빈도에 따른 인덱스값 부여
"""
X_train, y_train

(array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
        list([1, 194,

In [49]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

1641221/1641221 [==============================] - 0s 0us/step


'<sos> this film was just brilliant casting location scenery story'

전처리도 모델에 포함시키는 방법

In [52]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
print(datasets.keys())

/Users/gahyunson/opt/anaconda3/envs/mywork/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-09 23:38:49.786606: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 100%|██████████| 1/1 [00:14<00:00, 14.81s/ url]


Dataset imdb_reviews downloaded and prepared to /Users/gahyunson/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
dict_keys([Split('train'), Split('test'), Split('unsupervised')])


In [54]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300) # 각 데이터에서 300글자만 뽑는다
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ") # <br/> -> 공백 
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ") # 알파벳 제외한 것 -> 공백 
    X_batch = tf.strings.split(X_batch) # 공백 기준으로 나눠서 단어 단위로 처리
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch # tensor 타입으로 변환, padding 하여 리뷰 길이 맞추기 

# 어휘 사전 구축 
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))
        
preprocess(X_batch, y_batch)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


(<tf.Tensor: shape=(8, 60, 1), dtype=string, numpy=
 array([[[b'Red'],
         [b'Eye'],
         [b'is'],
         [b'not'],
         [b'the'],
         [b'kind'],
         [b'of'],
         [b'movie'],
         [b"that's"],
         [b'going'],
         [b'to'],
         [b'win'],
         [b'the'],
         [b'Palme'],
         [b"D'or"],
         [b'but'],
         [b'Wes'],
         [b'Craven'],
         [b'has'],
         [b'never'],
         [b'been'],
         [b'that'],
         [b'kind'],
         [b'of'],
         [b'director'],
         [b'anyway'],
         [b'and'],
         [b'his'],
         [b'branding'],
         [b'is'],
         [b'a'],
         [b'good'],
         [b'indication'],
         [b'of'],
         [b'what'],
         [b'a'],
         [b'film'],
         [b'goer'],
         [b'can'],
         [b'expect'],
         [b'The'],
         [b'fact'],
         [b'that'],
         [b'Red'],
         [b'Eye'],
         [b'is'],
         [b'a'],
         [b'tight'],

In [55]:
vocabulary.most_common()[:3] # 자주 등장하는 3개 

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

가장 자주 등장하는 단어로 정렬하여 10000개 단어만 남기고 삭제

In [56]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [57]:
# 인덱스값이 부여된 사전에서
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
# 원하는 텍스트를 인덱스 값으로 변환 
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

22
12
11
10000


In [59]:
words = tf.constant(truncated_vocabulary) # 사전의 단어들 
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64) # 사전 단어 길이만큼 range 값 
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids) # 단어와 인덱스값 매칭
num_oov_buckets = 1000 # 
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [65]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

tft.compute_and_apply_vocabulary() 함수에서 어휘 사전 사용 가능 

In [66]:
# train data preprocess 함수 적용하여 문장을 단어 단위로 변경
train_set = datasets["train"].batch(32).map(preprocess)

def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

# preprocess된 train data
train_set = train_set.map(encode_words).prefetch(1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [67]:
embed_size = 128
model = keras.models.Sequential([
    # Embedding layer. output shape : [batch size, time step, embedding size] 3D tensor
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # 마스킹 설정해서 패딩 토큰 알려줌 
                           input_shape=[None]), 
    # 2개 GRU layer
    # sequence, time step return
    keras.layers.GRU(128, return_sequences=True),
    # 타임 스텝의 출력만 return
    keras.layers.GRU(128),
    # 긍정/부정 
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
782/782 [==============================] - 184s 215ms/step - loss: 0.5317 - accuracy: 0.7266


## 2.1 마스킹
모델이 패딩 토큰을 무시하며 학습해야한다. 패딩 토큰을 무시하도록 모델에게 알려줄 수 있다.    
Embedding layer에서 mask_zero=True 매개변수를 주면된다.    
Embedding layer에서 설정하면 이후 모든 층에서 패딩 토큰을 무시한다.

Embedding layer에서 ...    
- mask tensor를 만든다. input and output size가 같은 텐서이다.    
- ID 0 - False, 나머지 - True.
- All layer에 타임 스텝 차원이 유지되면 자동으로 전파됨.
- 예를들어 GRU 층에서 sequence를 반환하지 않을 때는 마스크 텐서가 전달되지 않음

마스킹을 사용하려면 ...
- 마스킹을 지원하는 층이어야 한다
- 혹은 사용자 정의 층에서는 call() method에 mask 매개변수를 추가한다
- 생성자에서 self.supports_making=True 로
- Embedding이 시작 층이 아니면 keras.layers.Masking layer 사용 가능
- sequential model 에 가장 잘 맞음
- 복잡하 모델에서는 좀 ... -> 함수형 API, subclassing API에서 마스크 계산해서 다음 층에 전달 ...

